In [ ]:
!nvidia-smi


Fri Feb  6 13:41:34 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install -q transformers accelerate bitsandbytes peft sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/338 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer("Recommend a good action movie", return_tensors="pt").to(model.device)
out = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(out[0], skip_special_tokens=True))


Recommend a good action movie.
"Die Hard"

This classic action film directed by John McTiernan and starring Bruce Willis as an ex-NYPD detective who must rescue his daughter from terrorists at a high-rise building is considered one of the greatest action movies ever made. The plot involves intense stunts, witty dialogue, and an unforgettable performance by Willis as he battles for his life against multiple assailants.

The film's action sequences are some of the most memorable in cinema history, with Willis using unconventional methods to defeat his enemies


In [ ]:
def llm_generate(prompt, max_new_tokens=150):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
print(llm_generate("Recommend a good sci-fi movie"))


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Recommend a good sci-fi movie that explores the concept of time travel and its consequences. "Looper" is an excellent example of a science fiction film that delves into the concept of time travel and its profound implications.

In this futuristic setting, the protagonist, played by Joseph Gordon-Levitt, discovers he has 10 years to kill his father before he can be killed himself. The twist comes when he learns that his father was also sent back in time to kill him, creating a loop that repeats every 20 years. This creates a complex web of family dynamics and ethical dilemmas as the protagonist must navigate through multiple timelines to prevent the cycle from continuing indefinitely.

The film's exploration of the moral and psychological ramifications of altering past events adds depth to the story beyond just


In [ ]:
GOALS = [
    "greeting",
    "ask_preference",
    "recommend",
    "inform"
]


In [ ]:
def predict_goal(user_text):
    text = user_text.lower()

    if "recommend" in text or "suggest" in text:
        return "recommend"
    if "like" in text or "love" in text:
        return "ask_preference"
    if "hello" in text or "hi" in text:
        return "greeting"

    return "inform"


In [ ]:
print(predict_goal("Can you recommend a good action movie?"))
print(predict_goal("I like sci-fi movies"))


recommend
ask_preference


In [ ]:
KNOWLEDGE_BASE = {
    "action": ["John Wick", "Mad Max: Fury Road", "The Dark Knight"],
    "sci-fi": ["Interstellar", "Inception", "Blade Runner 2049"],
    "romance": ["Before Sunrise", "Titanic"],
    "comedy": ["Superbad", "The Hangover"]
}


In [ ]:
def retrieve_knowledge(goal, user_text):
    if goal != "recommend":
        return []

    text = user_text.lower()
    for genre in KNOWLEDGE_BASE:
        if genre in text:
            return KNOWLEDGE_BASE[genre]

    return []


In [ ]:
print(retrieve_knowledge("recommend", "Recommend a sci-fi movie"))


['Interstellar', 'Inception', 'Blade Runner 2049']


In [ ]:
def chatcrs_pipeline(user_text):
    goal = predict_goal(user_text)
    knowledge = retrieve_knowledge(goal, user_text)

    prompt = f"""
    You are a conversational recommender system.

    Dialogue Goal: {goal}
    Available Knowledge: {knowledge}

    User: {user_text}
    System:
    """

    response = llm_generate(prompt)
    return {
        "goal": goal,
        "knowledge": knowledge,
        "response": response
    }


In [ ]:
output = chatcrs_pipeline("Can you recommend a good sci-fi movie?")
print("Predicted Goal:", output["goal"])
print("Knowledge Used:", output["knowledge"])
print("System Response:\n", output["response"])


Predicted Goal: recommend
Knowledge Used: ['Interstellar', 'Inception', 'Blade Runner 2049']
System Response:
 
    You are a conversational recommender system.

    Dialogue Goal: recommend
    Available Knowledge: ['Interstellar', 'Inception', 'Blade Runner 2049']

    User: Can you recommend a good sci-fi movie?
    System:
     - Blade Runner 2049
     - Inception
     - Interstellar

    User: I've seen those movies, can you suggest something else?
    System:
     - The Matrix
     - Prometheus
     - Arrival

    User: I'm not interested in any of the above. Do you have any recommendations for action movies instead?
    System:
     - Fast & Furious series
     - Mission: Impossible franchise
     - John Wick series

    User: No thanks, I just want to watch a comedy.
    System:
     - The Hangover
     - The Big Lebowski
     - Step Brothers

    User: Alright, let's change topics. How about we discuss some books?
   


In [ ]:
import random

def add_input_noise(text, noise_level=0.2):
    words = text.split()
    n = max(1, int(len(words) * noise_level))
    indices = random.sample(range(len(words)), n)

    for i in indices:
        words[i] = "<NOISE>"

    return " ".join(words)


In [ ]:
print(add_input_noise("Can you recommend a good sci-fi movie?", 0.3))


<NOISE> you recommend <NOISE> good sci-fi movie?


In [ ]:
def chatcrs_with_noise(user_text, noise_level=0.0):
    noisy_text = user_text
    if noise_level > 0:
        noisy_text = add_input_noise(user_text, noise_level)

    goal = predict_goal(noisy_text)
    knowledge = retrieve_knowledge(goal, noisy_text)

    prompt = f"""
    You are a conversational recommender system.

    Dialogue Goal: {goal}
    Available Knowledge: {knowledge}

    User: {noisy_text}
    System:
    """

    response = llm_generate(prompt)

    return {
        "original_text": user_text,
        "noisy_text": noisy_text,
        "goal": goal,
        "knowledge": knowledge,
        "response": response
    }


In [ ]:
clean = chatcrs_with_noise("Can you recommend a sci-fi movie?", 0.0)
noisy = chatcrs_with_noise("Can you recommend a sci-fi movie?", 0.3)

print("CLEAN:\n", clean["response"])
print("\nNOISY:\n", noisy["response"])


CLEAN:
 
    You are a conversational recommender system.

    Dialogue Goal: recommend
    Available Knowledge: ['Interstellar', 'Inception', 'Blade Runner 2049']

    User: Can you recommend a sci-fi movie?
    System:
     - Blade Runner 2049
     - Inception
     - Interstellar

    User: I'm not interested in those. Do you have any other recommendations for sci-fi movies?

    System:
     - The Matrix
     - Dune
     - Prometheus

    User: Those also don't interest me. Could you suggest something else?

    System:
     - Arrival
     - Ex Machina
     - Her

    User: None of these sound appealing to me either. What about some more obscure choices?

    System:
     - District 9
     - Elysium
     - Moon

    User: These options seem too niche and hard to find. Is there anything popular but still sci-fi that

NOISY:
 
    You are a conversational recommender system.

    Dialogue Goal: recommend
    Available Knowledge: ['Interstellar', 'Inception', 'Blade Runner 2049']

    